In [ ]:
#!/usr/bin/env python3
"""
    File Name: Feature Map Visualiser.ipynb
    Author: Thomas Frew
    Date created: 08/01/2023
    Date last modified: 24/01/2023
    Python Version: 3.11
    
    Reads "Dataset.csv", training data for a computer vision AI.
    This can be in one of two forms:

        - 2D: Images are labelled with coordinates, defining the center of some object of interest
        - 4D: Images are labelled with coordinates, defining the bounding box around some object of interest

    This data is then used with an existing CNN, to visualise
    that CNN's feature maps. Feature maps represent how the
    AI "sees" an image.
"""

In [ ]:
# Import tensorflow for machine learning
import tensorflow as tf
from tensorflow.keras import models

# Import pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Import cv2 for image manipulation
import cv2

# Import Matplotlib for plotting
import matplotlib.pyplot as plt

# Import random for rnadomisation
import random

In [ ]:
# Load the training dataset. This can be generated with "2D Dataset Generator.ipynb or 4D Dataset Generator.ipynb"
dataset = pd.read_csv("Dataset.csv")

# Shuffle the dataset
dataset = dataset.sample(frac=1).reset_index(drop=True)

# Store the number of dimensions in the training data
output_dim = len(dataset.columns) - 1

# If the training dataset is 2D, then we are tracking center points
if (output_dim == 2):
    dataset.columns = ["Filename","x","y"]
    
    # Visualise the first 5 rows of the dataset
    print("The first 5 rows of your dataset are:")
    print(dataset.head())
    
# If the training dataset is 4D, then we are tracking bounding boxes
elif (output_dim == 4):
    dataset.columns = ["Filename","x1","y1","x2","y2"]

    # Visualise the first 5 rows of the dataset
    print("The first 5 rows of your dataset are:")
    print(dataset.head())
    
# Otherwise, something has gone wrong
else:
    print("Input data not in 2D (center-point) or 4D (bounding box) format.")

In [ ]:
# Split the image filenames (col 0) from their coordinate data (cols 1-2 or 1-4)
coords = dataset.copy()
file_names = coords.pop('Filename')

In [ ]:
# Load each file as its RGB image data
image_data = []

for fname in file_names:
    image_data.append(cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB))
    
# Store the length and width of each image
input_len, input_wid, channels = image_data[0].shape
    
# Report the image shape data
print(f"Your images are {input_len} by {input_wid} with {channels} color channels.")

In [ ]:
# Load the AI model created in "CNN Trainer.ipynb"
model = models.load_model("Model")

In [ ]:
# Visualise the model
print("The model's structure is:")
print(model.summary())

In [ ]:
# Convert the training images and labels to numpy arrays
image_data = np.asarray(image_data)
coords = np.asarray(coords)

In [ ]:
# Split up data into training and testing with a 9:1 ratio
datapoints = len(coords)
split_index = int(datapoints*0.1)

train_data = image_data[split_index:]
test_data = image_data[:split_index]

train_labels = coords[split_index:]
test_labels = coords[:split_index]

In [ ]:
# Store and show the names of layers in the CNN
layer_names = [layer.name for layer in model.layers]

print("The CNN's layer are:")
print(layer_names)

In [ ]:
# Create feature map model from CNN I/O
layer_outputs = [layer.output for layer in model.layers]
feature_map_model = tf.keras.models.Model(model.input, layer_outputs)

In [ ]:
# Ask the user to input the maximum number of feature maps they want to view, per layer
max_consecutive = int(input("How many feature maps would you like to view on each layer? (Reccomended: 3-7) \n>> "))

# Select a random image to process through the CNN
mapped_image = test_data[random.randrange(split_index) - 1]
mapped_image = np.expand_dims(mapped_image, axis=0)

# Create the feature maps for that particular image
feature_maps = feature_map_model.predict(mapped_image)

# Find the number of convolutional layers
conv_layers = 0;
for layer_name, feature_map in zip(layer_names, feature_maps):  
    if len(feature_map.shape) == 4:
        conv_layers += 1;

# Create a plot to display CNN feature maps
fig, axes = plt.subplots(conv_layers, 1, figsize=(15, 15)) 

# Layer index iterator
layer_i = 0

# Visualise all "interesting" (non-empty) feature maps
for layer_name, feature_map in zip(layer_names, feature_maps):  
    if len(feature_map.shape) == 4:
        
        # Define a stack of images in the same layer
        image_stack = []
        
        # Number of images in the layer
        k = feature_map.shape[-1]  
        
        # Counter for the number of "interesting" feature maps
        accepted = 0
        
        # Feature map iterator
        imap_i = 0
        
        # We collect interesting feature maps until we have enough or we run out of maps
        while (accepted < max_consecutive and imap_i < k):
            feature_image = feature_map[0, :, :, imap_i]
            maximum = np.max(feature_image)
            mean = np.mean(feature_image)
            
            if (mean > 0):
                feature_image *= 255/(maximum+1)
                image_stack.append(feature_image)
                
                accepted += 1
                
            imap_i += 1

        # Show our set of selected feature maps
        axes[layer_i].set_title(layer_name)
        axes[layer_i].grid(False)
        axes[layer_i].imshow(np.concatenate(image_stack, axis=1))
        layer_i += 1